In [2]:
import pandas as pd
import numpy as np
##visuals
import matplotlib.pyplot as plt
import seaborn as sns

## other
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

In [21]:
df = pd.read_csv('to_machinelearning.csv')
test = pd.read_csv('clean_test.csv')

In [22]:
ml = df.copy()

In [48]:
ml.head()

,HomePlanet,CryoSleep,Cabin_Deck,Cabin_Side,Cabin_DS,Destination,Age,VIP,RoomService,Spa,VRDeck,Total_Expenses,Transported
0,2,0,2,1,1,1,39.0,False,0.0,0.0,0.0,0.0,False
1,1,0,6,2,2,1,24.0,False,109.0,549.0,44.0,702.0,True
2,2,0,1,2,3,1,58.0,True,43.0,6715.0,49.0,6807.0,False
3,2,0,1,2,3,1,33.0,False,0.0,3329.0,193.0,3522.0,False
4,1,0,6,2,2,1,16.0,False,303.0,565.0,2.0,870.0,True


### Lets now do the data preprocessing

In [24]:
ml.drop(['PassengerId','Name','Passenger_group','Passenger_No','Cabin','Age_Catg','FoodCourt','ShoppingMall'],axis=1,inplace=True)

In [26]:
ml['HomePlanet'] = ml['HomePlanet'].map({'Earth':1,'Europa':2,'Mars':3})

In [28]:
ml['CryoSleep'] = ml['CryoSleep'].map({False:0,True:1})

In [36]:
ml['Cabin_Deck'] = ml['Cabin_Deck'].map({'A':1,'B':2,'C':3,'D':4,'E':5,'F':6,'G':7,'T':8})

In [39]:
ml['Cabin_Side'] = ml['Cabin_Side'].map({'P':1,'S':2})

In [44]:
dslist = ml['Cabin_DS'].unique().tolist()
dsid = np.arange(1,len(dslist)+1)
ml['Cabin_DS'].replace(dslist,dsid,True)

In [46]:
ml['Destination'].unique()

array(['TRAPPIST-1e', 'PSO J318.5-22', '55 Cancri e'], dtype=object)

In [47]:
dest_list = ml['Destination'].unique().tolist()
dest_id = np.arange(1,len(dest_list)+1)
ml['Destination'].replace(dest_list,dest_id,True)